In [1]:
# Common default imports
from pprint import pprint
from model_manager import LDAManager
from main.lda.config import LdaGeneratorConfig
from gensim import corpora
import pandas as pd
from gensim.corpora import Dictionary
# Plotting utility for the LDA models
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Stop words identified from the next blocks
stop_words = ['game', 'play', '<game_name>']

# Pre Processing Step
We want to process the data for LDA like:
- Keep NOUNS only as they are the elements that mostly hold aspect value
- Split reviews in sentences (Avoid topic modelling in a too broad sense) (https://aclanthology.org/N10-1122.pdf)

In [ ]:
# LDA does not benefit from repeated words among many documents. So we should clean them as they are stopwords.
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
ds = pd.read_csv(corpus_path)['comments'].apply(lambda x: x.split(' '))

dictionary = corpora.Dictionary(ds)
doc_frequency = {}

for index in range(len(dictionary)):
    doc_frequency[dictionary.get(index)] = len(ds[[dictionary.get(index) in x for x in ds]])
pd.DataFrame.from_dict(doc_frequency, orient='index')[0].map(lambda x: x / len(ds)).sort_values(ascending=False)

Are 'game'(40%), 'play'(20%) and '\<game_name\>'(11%) stopwords? <br>
For sure 'game' is! What about the other two? I believe they bring no context.
We remove them!

### Default: Sentences slit

In [6]:
# 80k
corpus_path = "../dataset/output/default_sentences/pre_processed.80k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences_80k"), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/68390 [00:00<?, ?it/s]

{'coherence': [-17.177862702834112,
               -17.02777543298526,
               -17.290263201442876,
               -16.936958294632383],
 'perplexity': -8.588481933552947,
 'topn': [3, 5, 10, 20]}


In [7]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
# Render the plot
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.190883  0.002691       1        1  10.265110
10     0.143829  0.116062       2        1   9.961937
4     -0.048755  0.034062       3        1   7.992255
2     -0.090078  0.035395       4        1   7.733778
13     0.051645  0.040787       5        1   7.196578
11     0.159590  0.037115       6        1   7.019158
5     -0.059096 -0.064933       7        1   6.711771
7     -0.066562  0.026106       8        1   6.662826
0      0.067464 -0.017909       9        1   6.654468
9      0.039191 -0.040358      10        1   6.594026
8     -0.029328 -0.057865      11        1   6.337182
3     -0.049830 -0.063875      12        1   5.786575
12    -0.034708  0.129836      13        1   5.625053
6      0.107520 -0.177115      14        1   5.459282, topic_info=           Term         Freq        Total Category  logprob  loglift
57         card  2241.000000  2241.000000  Default  30.0000  30.0000
258       great  1187.000000  1187.000000  Default  29.0000  29.0000
79         easy   644.000000   644.000000  Default  28.0000  28.0000
195        dice   521.000000   521.000000  Default  27.0000  27.0000
8        player  2470.000000  2470.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
7          love    42.924259   648.353831  Topic14  -5.2775   0.1929
286  experience    38.811085   352.073171  Topic14  -5.3783   0.7027
17         nice    38.186116   538.036268  Topic14  -5.3945   0.2624
33        board    38.142863   624.151115  Topic14  -5.3956   0.1128
118         fun    39.012879  1550.594174  Topic14  -5.3731  -0.7746

[1161 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
111       1  0.042018  <number>
111       2  0.028012  <number>
111       3  0.028012  <number>
111       4  0.084037  <number>
111       5  0.065362  <number>
...     ...       ...       ...
700      12  0.503348     young
3807      2  0.954814   youtube
5852     14  0.904323        yr
7507      5  0.840289      zany
7420      6  0.878548       zee

[4388 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 5, 3, 14, 12, 6, 8, 1, 10, 9, 4, 13, 7])

Now the bigger dataset for 310k

In [2]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, True, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from fs
Model not found. Making a new one.
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

{'coherence': [-14.345296267821746,
               -13.742494545430045,
               -14.779922291032763,
               -15.116752158504827],
 'perplexity': -8.19623024177193,
 'topn': [3, 5, 10, 20]}


In [3]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
# Render the plot
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.231924 -0.006110       1        1  9.162896
12    -0.100420  0.044382       2        1  8.825932
7      0.203365 -0.140904       3        1  8.540160
2      0.080816  0.064884       4        1  8.042860
1     -0.014630  0.184309       5        1  7.608133
8     -0.158234 -0.158961       6        1  7.512413
9      0.036677  0.174920       7        1  7.412632
10    -0.164390 -0.123739       8        1  7.394272
6      0.072340 -0.155483       9        1  6.960229
11     0.064044  0.096032      10        1  6.736107
0     -0.080141  0.073942      11        1  5.758742
5      0.201921 -0.064437      12        1  5.673783
4      0.051864  0.063850      13        1  5.270739
3      0.038715 -0.052686      14        1  5.101103, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  7263.000000  7263.000000  Default  30.0000  30.0000
147    good  6269.000000  6269.000000  Default  29.0000  29.0000
142    well  2448.000000  2448.000000  Default  28.0000  28.0000
133    time  4748.000000  4748.000000  Default  27.0000  27.0000
209    love  2371.000000  2371.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
840   solid   109.440791   623.082266  Topic14  -5.5117   1.2364
128    make   126.310025  1822.839299  Topic14  -5.3684   0.3063
147    good   129.113918  6269.552848  Topic14  -5.3464  -0.9071
24   design   114.543730  1164.904720  Topic14  -5.4661   0.6563
158    need   115.560843  2214.071117  Topic14  -5.4573   0.0229

[1083 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.056123  <number>
694       2  0.017460  <number>
694       4  0.001247  <number>
694       5  0.710891  <number>
694       6  0.004989  <number>
...     ...       ...       ...
207      11  0.018825      year
207      13  0.935478      year
739      10  0.973116         z
8626     10  0.991476       zee
8117     10  0.923992       zen

[4588 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 8, 3, 2, 9, 10, 11, 7, 12, 1, 6, 5, 4])

### NOUN-sentences ds
Filter out any non noun from dataset using the POS tagged processing pipleine

In [ ]:
from pre_processing import extract_pos_ds
import pandas as pd

print("Creating the __noun filtered ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

print("Creating the __noun filtered test ds:")
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.test.csv"
store_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"
extract_pos_ds(pd.read_csv(corpus_path)['comments'], "__noun", store_path)
print("ds created under: " + store_path)

Now we check if the NOUN only approach works better. <br>
Take nouns only of the POS tagged ds:

In [ ]:
corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged_sentence_level/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

In [14]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.233209 -0.005561       1        1  9.198038
12    -0.101986  0.047204       2        1  8.843128
7      0.202171 -0.143089       3        1  8.535858
2      0.080518  0.066422       4        1  8.031112
1     -0.013881  0.184347       5        1  7.622198
8     -0.160835 -0.157964       6        1  7.480469
9      0.038786  0.176612       7        1  7.444389
10    -0.167064 -0.120826       8        1  7.349323
6      0.072377 -0.157634       9        1  6.967530
11     0.065899  0.096497      10        1  6.704844
0     -0.078668  0.072741      11        1  5.783340
5      0.202150 -0.065068      12        1  5.691101
4      0.053988  0.062474      13        1  5.253688
3      0.039752 -0.056154      14        1  5.094982, topic_info=       Term         Freq        Total Category  logprob  loglift
174    card  7263.000000  7263.000000  Default  30.0000  30.0000
147    good  6259.000000  6259.000000  Default  29.0000  29.0000
142    well  2449.000000  2449.000000  Default  28.0000  28.0000
133    time  4740.000000  4740.000000  Default  27.0000  27.0000
209    love  2398.000000  2398.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
146    come   129.759035  1385.767935  Topic14  -5.3402   0.6086
128    make   133.108429  1822.262821  Topic14  -5.3147   0.3602
24   design   116.141080  1174.316738  Topic14  -5.4511   0.6633
147    good   117.176215  6259.676425  Topic14  -5.4422  -1.0013
158    need   111.652030  2225.565038  Topic14  -5.4905  -0.0155

[1080 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.055129  <number>
694       2  0.017541  <number>
694       4  0.001253  <number>
694       5  0.717934  <number>
694       6  0.005012  <number>
...     ...       ...       ...
207      11  0.019068      year
207      13  0.937261      year
739      10  0.952437         z
8626     10  0.984800       zee
8117     10  0.979445       zen

[4552 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 13, 8, 3, 2, 9, 10, 11, 7, 12, 1, 6, 5, 4])

Now that the dataset was built and stored train an LDA on it:

In [4]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"

model_manager = LDAManager.from_config(LdaGeneratorConfig(name="default_sentences"), stop_words)
model = model_manager.get_model(corpus_path, False, True)
res = model_manager.evaluate(test_corpus_path)

pprint(res)

Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-8.206318645320582,
               -8.092144166145504,
               -8.572682666308848,
               -9.292421386724754],
 'perplexity': -7.193918552296681,
 'topn': [3, 5, 10, 20]}


In [5]:
dictionary: Dictionary = model.id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(model, texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)

vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.186977 -0.027139       1        1  14.165697
6      0.151868 -0.040687       2        1   9.435898
1      0.104165  0.129464       3        1   8.514341
2     -0.047553  0.033814       4        1   8.384107
5     -0.155608 -0.039763       5        1   7.256713
7      0.017078  0.018696       6        1   7.113230
3      0.000196  0.078796       7        1   6.845400
9     -0.112893 -0.054376       8        1   6.400776
10    -0.085526  0.059666       9        1   6.359796
4     -0.084226  0.107590      10        1   5.981675
12     0.077980  0.076606      11        1   5.432623
8     -0.096928  0.018351      12        1   4.824361
11     0.150969 -0.162208      13        1   4.671468
13    -0.106498 -0.198811      14        1   4.613913, topic_info=          Term          Freq         Total Category  logprob  loglift
9       player  20121.000000  20121.000000  Default  30.0000  30.0000
87        card  17229.000000  17229.000000  Default  29.0000  29.0000
156       dice   4384.000000   4384.000000  Default  28.0000  28.0000
83   expansion   3693.000000   3693.000000  Default  27.0000  27.0000
382     action   4687.000000   4687.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
56       board    159.242220   5019.828217  Topic14  -5.1087  -0.3746
386       feel    128.018954    903.564631  Topic14  -5.3270   1.1219
9       player    167.059296  20121.594383  Topic14  -5.0608  -1.7151
83   expansion    145.204889   3693.798325  Topic14  -5.2010  -0.1602
111        way    141.145856   5081.450331  Topic14  -5.2294  -0.5075

[1133 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
3945      14  0.927588        aa
293        1  0.200203   ability
293        2  0.061386   ability
293        3  0.263682   ability
293        4  0.018137   ability
...      ...       ...       ...
10655      2  0.925629  ziggurat
1723       3  0.931097    zombie
1723       4  0.005060    zombie
1723       8  0.005060    zombie
1723      12  0.055663    zombie

[4946 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 3, 6, 8, 4, 10, 11, 5, 13, 9, 12, 14])

Following the experiments I decided to follow the path of nouns only composition document. <br>
The hyperparameters tuning is now the next step.

Noun only on sentences model performs way worse, let's see if going for non LocalLDA but focusing on NOUNS only is viable:

# Hyperparameter tuning results applied
We found the best $K$ for our LDA models applied on the datasets. <br>
As stated in the tuning notebook the NOUN on sentences pipeline produced low coherence models reason for that approach was dropped.

### Default sentences
Best found $K \in \{7, 9\}$ <br>
Let's try training two models on the full data and compare them.

In [2]:
corpus_path = "../dataset/output/default_sentences/pre_processed.310k.csv"
test_corpus_path = "../dataset/output/default_sentences/pre_processed.310k.test.csv"

runs = [
    dict(config=LdaGeneratorConfig(name="sentences-K9", topics=9), results=[], model=None),
    dict(config=LdaGeneratorConfig(name="sentences-K11", topics=11), results=[], model=None)
]

In [3]:
for run in runs:
    model_manager = LDAManager.from_config(run['config'], stop_words)
    print(model_manager.config)
    run['model'] = model_manager.get_model(corpus_path)
    run['results'] = model_manager.evaluate(test_corpus_path)
    # Print the results just cause
    pprint(run['results'])

LdaGeneratorConfig(name='sentences-K9', topics=9, random_state=42, passes=10, alpha='symmetric', eta=None, output_folder='./output')
Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

{'coherence': [-16.814645681233134,
               -15.08919057759007,
               -14.764100315046967,
               -14.83918106863862],
 'perplexity': -8.022003243917984,
 'topn': [3, 5, 10, 20]}
LdaGeneratorConfig(name='sentences-K11', topics=11, random_state=42, passes=10, alpha='symmetric', eta=None, output_folder='./output')
Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/233897 [00:00<?, ?it/s]

{'coherence': [-13.430330864577495,
               -14.203485979641725,
               -14.612426153560664,
               -15.049984868299163],
 'perplexity': -8.116019899285954,
 'topn': [3, 5, 10, 20]}


Compare the two visualizations

In [4]:
pyLDAvis.enable_notebook()

In [5]:
dictionary: Dictionary = runs[0]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[0]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.134347  0.047909       1        1  18.922194
1     -0.154417 -0.093075       2        1  14.667473
2     -0.037124 -0.141974       3        1  11.123446
7      0.211939 -0.000649       4        1  10.780922
6      0.112372 -0.001080       5        1   9.756341
0     -0.136911  0.105369       6        1   9.333917
5      0.152258 -0.066940       7        1   9.016760
4     -0.049172 -0.075023       8        1   8.779990
3      0.035402  0.225462       9        1   7.618956, topic_info=      Term         Freq        Total Category  logprob  loglift
174   card  7388.000000  7388.000000  Default  30.0000  30.0000
2     like  7860.000000  7860.000000  Default  29.0000  29.0000
142   well  2428.000000  2428.000000  Default  28.0000  28.0000
70     fun  5269.000000  5269.000000  Default  27.0000  27.0000
147   good  6289.000000  6289.000000  Default  26.0000  26.0000
..     ...          ...          ...      ...      ...      ...
297  token   160.947514   467.631880   Topic9  -5.5272   1.5079
128   make   211.667321  1821.039103   Topic9  -5.2533   0.4224
147   good   252.024217  6289.601533   Topic9  -5.0788  -0.6426
158   need   202.156305  2194.964178   Topic9  -5.2992   0.1897
292    set   176.855980  1783.245177   Topic9  -5.4330   0.2637

[746 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.046415  <number>
694       2  0.662350  <number>
694       3  0.003763  <number>
694       4  0.001254  <number>
694       5  0.023835  <number>
...     ...       ...       ...
1430      6  0.006764     young
1430      7  0.818473     young
1430      8  0.175870     young
8117      6  1.008276       zen
1974      6  0.975870       zoo

[2610 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 3, 8, 7, 1, 6, 5, 4])

In [6]:
dictionary: Dictionary = runs[1]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[1]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.163064 -0.151431       1        1  14.630102
1      0.111834  0.148577       2        1  10.464167
2     -0.013668  0.074164       3        1   9.665917
9      0.051419  0.188785       4        1   9.080308
7     -0.226111 -0.044226       5        1   9.041011
6     -0.122876 -0.092900       6        1   8.557221
5     -0.207934  0.067634       7        1   8.305623
10     0.038064 -0.185585       8        1   8.154966
0      0.155213  0.018737       9        1   7.868996
4      0.000389  0.051970      10        1   7.807173
3      0.050607 -0.075726      11        1   6.424516, topic_info=        Term         Freq        Total Category  logprob  loglift
174     card  7330.000000  7330.000000  Default  30.0000  30.0000
133     time  4714.000000  4714.000000  Default  29.0000  29.0000
56      feel  4115.000000  4115.000000  Default  28.0000  28.0000
147     good  6233.000000  6233.000000  Default  27.0000  27.0000
142     well  2429.000000  2429.000000  Default  26.0000  26.0000
..       ...          ...          ...      ...      ...      ...
56      feel   185.066732  4115.656061  Topic11  -5.2171  -0.3568
147     good   177.813062  6233.588681  Topic11  -5.2570  -0.8119
146     come   145.396671  1366.334681  Topic11  -5.4583   0.5046
128     make   135.570089  1813.893381  Topic11  -5.5283   0.1513
172  overall   125.531949   616.040682  Topic11  -5.6052   1.1543

[887 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
694       1  0.048948  <number>
694       2  0.537169  <number>
694       3  0.005020  <number>
694       4  0.136802  <number>
694       5  0.001255  <number>
...     ...       ...       ...
207       7  0.001453      year
207       9  0.005813      year
207      10  0.622012      year
1430      7  0.985809     young
1430     10  0.006846     young

[3447 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 2, 3, 10, 8, 7, 6, 11, 1, 5, 4])

Of the two trained models configurations the one with lower variance yields the better model.<br>
The $K=9$ configuration while having a better average does not return the most valid resulting model. <br>
For increasing top-n the $K=11$ has a better score and surpasses the other model.

In [25]:
from gensim.models import LdaModel, LdaMulticore

model = LdaMulticore.load('./output/sentences-K11/sentences-K11.model')
# We work on the best in coherence only
m1: LdaModel = model
m1.print_topics(20, 20)

[(0,
  '0.070*"player" + 0.038*"expansion" + 0.023*"high" + 0.014*"base" + 0.014*"want" + 0.012*"count" + 0.011*"rate" + 0.011*"point" + 0.011*"good" + 0.011*"low" + 0.010*"buy" + 0.010*"board" + 0.010*"number" + 0.009*"include" + 0.009*"recommend" + 0.009*"turn" + 0.008*"money" + 0.007*"score" + 0.007*"room" + 0.007*"interaction"'),
 (1,
  '0.027*"rule" + 0.025*"player" + 0.014*"go" + 0.014*"time" + 0.013*"turn" + 0.013*"end" + 0.013*"look" + 0.012*"get" + 0.012*"rating" + 0.011*"start" + 0.011*"hour" + 0.011*"easy" + 0.010*"know" + 0.009*"couple" + 0.008*"scenario" + 0.008*"lose" + 0.008*"try" + 0.008*"forward" + 0.008*"<number>" + 0.007*"win"'),
 (2,
  '0.064*"like" + 0.061*"feel" + 0.037*"lot" + 0.036*"little" + 0.034*"bit" + 0.017*"fun" + 0.016*"player" + 0.013*"thing" + 0.012*"way" + 0.011*"different" + 0.009*"take" + 0.009*"think" + 0.009*"get" + 0.007*"actually" + 0.007*"interaction" + 0.006*"long" + 0.006*"interesting" + 0.006*"point" + 0.006*"work" + 0.005*"boring"'),
 (3,
  

In [ ]:
gold_aspects = ["luck", "bookkeeping", "downtime", "interaction", "bash", "complex/complicated", "misc"]
mapped_topics = {
    "0": "misc",  # Recommendation / Own / Want
    "1": "bash",  # Could also be downtime but bash seems to fit
    "2": "downtime",  # Seems noisy
    "3": "misc",  # Value (in terms of the game)
    "4": "misc",  # ???
    "5": "complex/complicated",  # Light + Family + Party -> Indicate complexity low or high
    "6": "bash",  # Directly act towards pushing down the player
    "7": "complex/complicated",  #
    "8": "bookkeeping",  # Resource management
    "9": "downtime",  # Temporal references and boring + ap
    "10": "interaction",  # Mechanics (I believe it to be interaction)
}

# With this mapping we can infer the aspect by label after classifying with LDA

Map to gold standards and/or recognize the associated aspect

### NOUN only
The best found $K=?$ <br>
Let's try training two models on the full data and compare them.


In [15]:
corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.csv"
test_corpus_path = "../dataset/output/pos_tagged/pre_processed.310k.noun_only.test.csv"

runs = [
    dict(config=LdaGeneratorConfig(name="noun-K7", topics=7), results=[], model=None),
    dict(config=LdaGeneratorConfig(name="noun-K9", topics=9), results=[], model=None),
]

In [8]:
for run in runs:
    model_manager = LDAManager.from_config(run['config'], stop_words)
    run['model'] = model_manager.get_model(corpus_path)
    run['results'] = model_manager.evaluate(test_corpus_path)
    # Print the results just cause
    pprint(run['results'])

Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-7.948833081292833,
               -6.802857928452987,
               -6.95858608877034,
               -7.416240587926887],
 'perplexity': -6.993845573618511,
 'topn': [3, 5, 10, 20]}
Generating a new compiled model from scratch
Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/195789 [00:00<?, ?it/s]

{'coherence': [-8.906816819996525,
               -7.345944142314569,
               -7.23757389847958,
               -8.497291086234405],
 'perplexity': -7.057565041995237,
 'topn': [3, 5, 10, 20]}


Visualize the results:

In [9]:
pyLDAvis.enable_notebook()

In [10]:
dictionary: Dictionary = runs[0]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[0]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.175843  0.039813       1        1  20.980051
6     -0.107931  0.073301       2        1  15.445445
2      0.041627 -0.000171       3        1  14.047615
3      0.031924  0.022279       4        1  13.642506
1     -0.089691 -0.060889       5        1  12.577611
5      0.212880  0.120912       6        1  11.801582
4      0.087034 -0.195245       7        1  11.505190, topic_info=           Term          Freq         Total Category  logprob  loglift
9        player  20255.000000  20255.000000  Default  30.0000  30.0000
87         card  16975.000000  16975.000000  Default  29.0000  29.0000
83    expansion   3673.000000   3673.000000  Default  28.0000  28.0000
382      action   4672.000000   4672.000000  Default  27.0000  27.0000
28         rule   6948.000000   6948.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
56        board    627.635462   5053.762564   Topic7  -4.6509   0.0764
26     mechanic    615.911342   5107.028770   Topic7  -4.6698   0.0471
34   collection    390.996159   1762.974140   Topic7  -5.1242   0.6563
17        thing    474.744175   3981.654740   Topic7  -4.9301   0.0357
223   character    366.773389   2061.119049   Topic7  -5.1881   0.4361

[585 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
293       1  0.267224  ability
293       2  0.082546  ability
293       3  0.036376  ability
293       4  0.137110  ability
293       5  0.412729  ability
...     ...       ...      ...
4896      7  0.975909       yr
3902      4  0.952559      zen
1723      3  0.010336   zombie
1723      5  0.981956   zombie
1723      7  0.005168   zombie

[1777 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 3, 4, 2, 6, 5])

In [11]:
dictionary: Dictionary = runs[1]['model'].id2word
# Load texts to
texts = pd.read_csv(test_corpus_path)['comments'].apply(lambda x: x.split(' '))

# Visualize the topics
vis = gensimvis.prepare(runs[1]['model'], texts.apply(lambda x: dictionary.doc2bow(x)).tolist(), dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.183257  0.003845       1        1  17.719847
6     -0.143092 -0.042164       2        1  12.515537
7     -0.007801 -0.099534       3        1  11.487340
1     -0.127579  0.075929       4        1  11.482424
2      0.027475 -0.048020       5        1  10.878318
3     -0.004436 -0.030198       6        1   9.654124
5      0.198359 -0.151033       7        1   9.496771
8      0.133277  0.135543       8        1   8.909846
4      0.107054  0.155632       9        1   7.855792, topic_info=           Term          Freq         Total Category  logprob  loglift
87         card  17088.000000  17088.000000  Default  30.0000  30.0000
9        player  20195.000000  20195.000000  Default  29.0000  29.0000
83    expansion   3691.000000   3691.000000  Default  28.0000  28.0000
156        dice   4439.000000   4439.000000  Default  27.0000  27.0000
97          fun   8146.000000   8146.000000  Default  26.0000  26.0000
..          ...           ...           ...      ...      ...      ...
178    gameplay    299.743275   2694.251943   Topic9  -5.0084   0.3480
4        choice    259.775231   2204.611952   Topic9  -5.1515   0.4054
1       edition    235.477339   1435.492000   Topic9  -5.2497   0.7363
342       power    221.675092   1493.936941   Topic9  -5.3101   0.6360
34   collection    221.880859   1774.680546   Topic9  -5.3092   0.4647

[757 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
293       1  0.208535  ability
293       2  0.074626  ability
293       3  0.071139  ability
293       4  0.384987  ability
293       5  0.030687  ability
...     ...       ...      ...
1723      4  0.870286   zombie
1723      5  0.015449   zombie
1723      7  0.030898   zombie
1723      8  0.077244   zombie
2675      2  0.994872      zoo

[2645 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 8, 2, 3, 4, 6, 9, 5])

The best in terms of coherence is the best found model as it also had lower variance and best overall average. <br>
$K=9$ was chosen as it better performed in top-3 but the first one is more robust.

Map to gold standards and/or recognize the associated aspect

In [14]:
from gensim.models import LdaModel, LdaMulticore

# We work on the best in coherence only
m1: LdaModel = runs[0]['model']
m1.print_topics(20, 20)

[(0,
  '0.037*"action" + 0.034*"card" + 0.028*"point" + 0.023*"building" + 0.019*"worker" + 0.019*"turn" + 0.019*"resource" + 0.017*"placement" + 0.012*"round" + 0.012*"player" + 0.012*"area" + 0.011*"way" + 0.011*"lot" + 0.011*"board" + 0.011*"engine" + 0.010*"end" + 0.009*"mechanic" + 0.009*"opponent" + 0.009*"order" + 0.009*"scoring"'),
 (1,
  '0.147*"card" + 0.027*"deck" + 0.026*"player" + 0.017*"luck" + 0.014*"hand" + 0.013*"filler" + 0.013*"gamer" + 0.012*"time" + 0.011*"co" + 0.010*"trick" + 0.009*"minute" + 0.009*"value" + 0.008*"way" + 0.008*"ability" + 0.008*"decision" + 0.008*"op" + 0.007*"draw" + 0.007*"round" + 0.007*"set" + 0.007*"character"'),
 (2,
  '0.058*"rule" + 0.028*"time" + 0.025*"player" + 0.019*"theme" + 0.018*"hour" + 0.016*"mechanic" + 0.016*"system" + 0.014*"scenario" + 0.011*"lot" + 0.009*"rating" + 0.009*"combat" + 0.008*"battle" + 0.008*"war" + 0.008*"rulebook" + 0.008*"wargame" + 0.007*"component" + 0.007*"book" + 0.007*"way" + 0.007*"copy" + 0.006*"gamep

In [ ]:
gold_aspects = ["luck", "bookkeeping", "downtime", "interaction", "bash", "complex/complicated", "misc"]
mapped_topics = {
    "0": "interaction",  # Strategy/Planning
    "1": "luck",  # Direct luck references as draw and cards and op + Ability (which can indicate low luck)
    "2": "downtime",  # Downtime seems most fitting
    "3": "complex/complicated",  # Strategy - Table
    "4": "misc",  # Components
    "5": "misc",  # Player Experience
    "6": "bash",  # I am not convinced over this one, could be interaction
}

# With this mapping we can infer the aspect by label after classifying with LDA

While NOUN processing was better in terms of coherence the overall feeling I get from the aspects is less understandable than the ones from the sentence level. <br>
It acutally does have less  noise and more conceptual separation.